In [1]:
# import libraries
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, TaskType

In [2]:
!pip install bitsandbytes

In [3]:
!pip install --upgrade datasets

In [4]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_compute_dtype= torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout= 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [6]:
data = load_dataset('openai/gsm8k', 'main', split='train[:200]')

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [7]:
def tokenize(batch):
  texts = [
      f"### Instruction:\n{instruction}\n### Response:\n{out}"
      for instruction , out in zip(batch['question'], batch['answer'])
  ]

  tokens = tokenizer(
      texts,
      padding = 'max_length',
      max_length = 256,
      truncation = True,
      return_tensors = 'pt'
  )

  tokens['labels'] = tokens['input_ids'].clone()
  return tokens

In [8]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
training_args = TrainingArguments(
    output_dir = './tinyllama-math-lora',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True,
    logging_steps= 20,
    save_strategy = 'epoch',
    report_to= 'none',
    remove_unused_columns= False,
    label_names = ['labels']
)

In [10]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [11]:
trainer.train()

Step,Training Loss
20,1.919000
40,0.811900
60,0.711400
80,0.626000
100,0.529500
120,0.458900
140,0.384400
160,0.301700
180,0.249100
200,0.189300


TrainOutput(global_step=600, training_loss=0.24247278680404027, metrics={'train_runtime': 1077.3518, 'train_samples_per_second': 9.282, 'train_steps_per_second': 0.557, 'total_flos': 1.4685722501971968e+16, 'train_loss': 0.24247278680404027, 'epoch': 46.16})

In [12]:
model.save_pretrained('./tinyllama-lora-tuned-adapter')
tokenizer.save_pretrained('./tinyllama-lora-tuned-adapter')

('./tinyllama-lora-tuned-adapter/tokenizer_config.json',
 './tinyllama-lora-tuned-adapter/special_tokens_map.json',
 './tinyllama-lora-tuned-adapter/tokenizer.model',
 './tinyllama-lora-tuned-adapter/added_tokens.json',
 './tinyllama-lora-tuned-adapter/tokenizer.json')